In [1]:
import modal
from egfr_binder_rd2.utils import hash_seq
from egfr_binder_rd2 import EGFR, EGFS
import pandas as pd


In [9]:


get_msa = modal.Function.lookup("simplefold", 'get_msa_for_binder')
a3m_from_template = modal.Function.lookup("simplefold", 'a3m_from_template')
fold_binder = modal.Function.lookup("simplefold", 'fold_binder')
update_metrics = modal.Function.lookup("simplefold", 'update_metrics_for_all_folded')
esm2_pll = modal.Function.lookup("esm2-inference", 'process_sequences')
update_pll_metrics = modal.Function.lookup("esm2-inference", 'update_pll_metrics')

# get fasta to run bindcraft stuff

In [11]:
df = pd.read_csv('/home/naka/code/BindCraft/outputs/EGFR_single_domain/final_design_stats.csv')

In [12]:
df = df.sort_values('Average_i_pTM', ascending=False)

In [13]:
binder_seqs = {}
for i, row in df.iterrows():
    binder_seq = row['Sequence']
    complex = f'{binder_seq}:{EGFR}'
    binder_seqs[hash_seq(complex)] = complex


In [15]:
# Save sequences to FASTA
with open('bindcraft_seqs.fasta', 'w') as f:
    for seq_hash, complex_seq in binder_seqs.items():
        f.write(f'>{seq_hash}\n{complex_seq}\n')


# run pll on last rounds stuff

In [5]:
df = pd.read_csv('/home/naka/code/egfr_binder_rd2/data/fold_df.csv', index_col=0)

In [11]:
binder_seqs = df['binder_sequence'].to_list()

In [12]:
result = esm2_pll.remote(binder_seqs)

In [13]:
metrics = update_pll_metrics.remote()

# other

In [14]:
binder_seqs = [
    'WVQLQESGGGLVQPGGSLRLSCAASGRTFSSYAMGWFRQAPGKQREFVAAIRWSGGYTYYTDSVKGRFTISRDNAKTTVYLQMNSLKPEDTAVYYCAATYLSSDYSRYALPQRPLDYDYWGQGTQVTVSSLE:LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCKLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLRGCPTNGHHHHHH',
    'QVQLQESGGGLVQPGGSLRLSCAASGRTFSSHAMGWFRQAPGKQREFVAAIRWSGGYTYYTDSVKGRFTISRDNAKTTVYLQMNSLKPEDTAVYYCAATYLSSDYSRYALPQRPLDYDYWGQGTQVTVSSLE:LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCKLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLRGCPTNGHHHHHH',
    'SVDEECPASYEGFCQNDGTCLYLEKLDRYACRCREGYIGERCEFRDLDYWLEQ',
    'DSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWENLEERLKEHRAKRLALLGPGPPGVVEKEKYKVSITEKVNPGGPATMPMTLTDSNGNKTTLTITVTPEGLEAIRKRRAGEKVKYTMTSTDTGDKFVLVDLDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLK',
    'DSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWENLEERLKEHRAKRLALL',
    'NSDSECPLSHDGYCLHDGVCMYIEALDKYACNCVVGYIGERCQYRDLKWWELR',
]

result = get_msa.remote(binder_seqs)


In [13]:
parent_binder_seq = 'SKEEEYYEEHQKLAKPVEELWEKLDELEKTGKLTGEHRPLVTEFRRLWSDAMVLIAMYMWYLEEVDKNPSEENRKKAQEYLEKVEEKKKEMEELLKKL'

In [ ]:
result = get_msa.remote(parent_binder_seq)

In [14]:
result = fold_binder.remote(binder_seqs=[parent_binder_seq])

NameError: name 'parent_binder_seq' is not defined

In [7]:
a3m_path = a3m_from_template.remote(binder_seq='SALDEWLASLDPQVGQDIRDYIEERQAE', parent_binder_seq=parent_binder_seq)

In [21]:
result

PosixPath('/colabfold_data/msa_results/83f160.a3m')

In [9]:

result = fold_binder.remote(binder_seq='SALDEWLASLDPQVGQDIRDYIEERQAE', parent_binder_seq=parent_binder_seq)

In [15]:
df = update_metrics.remote()

In [16]:
df

,seq_hash,binder_sequence,binder_length,target_sequence,target_length,model_number,binder_plddt,binder_pae,pae_interaction,ptm,i_ptm,binder_charged_fraction,binder_hydrophobic_fraction,parent_hash,parent_sequence,mutations
0,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,85.922264,5.186330,8.895448,0.86,0.92,0.207547,0.301887,NaN,NaN,NaN
1,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,2,84.365472,5.574172,9.162459,0.86,0.91,0.207547,0.301887,NaN,NaN,NaN
2,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,3,85.697547,5.311627,9.028248,0.85,0.91,0.207547,0.301887,NaN,NaN,NaN
3,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,4,84.229811,5.659277,9.556460,0.85,0.90,0.207547,0.301887,NaN,NaN,NaN
4,5b353a,NSYPGCPSSYDGYCLNGGVCMHIESLDSYTCNCVIGYSGDRCQTRD...,53,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,5,82.960566,5.779459,9.915783,0.83,0.90,0.207547,0.301887,NaN,NaN,NaN
5,eec3a5,SALDEWLASLDPQVGQDIRDYIEERQAE,28,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,33.138571,6.073788,22.274732,0.63,0.38,0.357143,0.285714,35530.0,SSLDEWLASLDPQVGQDIRDYIEERQAE,S2A
6,5c59d8,ASLDEWLASLDPQVGQDIRDYIEERQAE,28,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,32.548214,6.130702,22.365877,0.62,0.38,0.357143,0.285714,35530.0,SSLDEWLASLDPQVGQDIRDYIEERQAE,S1A
7,83f160,SKEEEYYEEHQKLAKPVEELWEKLDELEKTGKLTGEHRPLVTEFRR...,98,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,87.602449,2.957045,15.524411,0.62,0.91,0.479592,0.326531,NaN,NaN,NaN
8,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,79.039538,3.777522,16.788581,0.58,0.89,0.553846,0.292308,NaN,NaN,NaN
9,6f4ce9,AERMRRRVERIVEIHEEEAKEVLENLKKQGMKEEDLKCMEEYLEQD...,65,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,622,1,42.019231,7.973046,25.950806,0.60,0.25,0.569231,0.276923,f9aee7,AERMRRRFEHIVEIHEEWAKEVLENLKKQGSKEEDLKFMEEYLEQD...,"F8V,H10R,W18E,S31M,F38C"
